!pip install beautifulsoup4

!pip install lxml

!pip install xlsxwriter

In [61]:
!pip install schedule

  Obtaining dependency information for schedule from https://files.pythonhosted.org/packages/f0/1a/e5a65c08d708ee4c002f3f938ba69b2282af478755700c40f037eacc92ef/schedule-1.2.1-py2.py3-none-any.whl.metadata


In [54]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime



In [2]:
url = "https://store.steampowered.com/"
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<html class="responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Welcome to Steam</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://store.cloudflare.steamstatic.com/public/shared/css/motiva_sans.css?v=2C1Oh9QFVTyK&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
<link href="https://store.cloudflare.steamstatic.com/public/shared/css/shared_global.css?v=A9Y8E9x2QxEv&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
<link href="https://store.cloudflare.steamstatic.com/public/shared/css/buttons.css?v=Grv-5yNFuEfg&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
<link href="https://store.cloudflare.steamstatic.com/public/css/v6/store.css?v=jLYTEsuO11IC&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet"

In [13]:
soup.title.text

'Welcome to Steam'

In [18]:
array = []

In [58]:
def stream_scraper():
    current_time = datetime.now().strftime('%H-%M-%S')
    url = "https://store.steampowered.com/"
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    soup
    div_tag = soup.find_all('a', class_='sale_capsule TopSellerCapsule')
    for d in div_tag:
        game_array = []
        string = d['href'].split("/")
        game_name = string[5]
        game_array.append(game_name)
        div_discount_div = d.find('div', class_='discount_block discount_block_inline')
        if div_discount_div is not None:
            div_original_price = div_discount_div.find('div', class_='discount_original_price')
            if div_original_price is not None:
                if div_original_price.text != "":
                    original_price = div_original_price.text
                    game_array.append(original_price)
                else:
                    div_original_price = ""
                div_discounted_div = div_discount_div.find('div', class_='discount_final_price')
                if div_discounted_div.text != "":
                    discounted_price = div_discounted_div.text
                    game_array.append(discounted_price)
                else:
                    discounted_price = ""
                div_discount_div = div_discount_div.find('div', class_='discount_pct')
                if div_discount_div.text != "":
                    discount = div_discount_div.text
                    game_array.append(discount)
                else:
                    discount = ""
                array.append(game_array)
    df = pd.DataFrame(array,columns=['Game','Original Price','Discounted Price','Discount'])
    df = df.sort_values(by='Discount', ascending=False)
    df = df.reset_index(drop=True)
    with pd.ExcelWriter('steam_sale'+str(current_time)+'.xlsx', engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Sheet1', index=False)

In [59]:
stream_scraper()

In [ ]:
import schedule
import time


# Schedule the task to run every minute
schedule.every(1).minutes.do(stream_scraper)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)
